In [2]:
import requests
import json
import pandas as pd
from functools import reduce
from pandas.io.json import json_normalize

URL = 'http://real_search:8983/solr/dbnews/select'
T_URL = 'http://search:8983/solr/news/select'
def get_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result


def get_tot_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(T_URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [32]:
week1 = ['2017-03-27','2017-04-02']
week2 = ['2017-04-03','2017-04-09']
week3 = ['2017-04-10','2017-04-16']
week4 = ['2017-04-17','2017-04-23']
week5 = ['2017-04-24','2017-04-30']

week = [
    ['2017-03-27','2017-04-02'], 
    ['2017-04-03','2017-04-09'],
    ['2017-04-10','2017-04-16'],
    ['2017-04-17','2017-04-23'],
    ['2017-04-24','2017-04-30']   
       ]

from_dt = '2017-04-01'
to_dt = '2017-04-01'

In [33]:
df_result = json_normalize(get_tot_search_result('', from_dt, to_dt))

In [34]:
df_result['tot'] = df_result['n_shares'].fillna(0) + df_result['o_shares'].fillna(0) 

KeyError: 'n_shares'

In [35]:
df_result['title'] = df_result['title'].str.replace(r'\([가-힣=]+\)|\[[a-z0-9가-힣]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')

In [36]:
df_result['title']=df_result['title'].fillna('')
documents = df_result['title'].tolist()
#pos = lambda d: ['/'.join(p) for p in t.pos(d)]
#texts_ko = [pos(doc) for doc in documents]
#print(texts_ko[0])

In [37]:
from konlpy.tag import Mecab
t=Mecab()
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    return [pos[0] for pos in t.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    #if pos[1] in ['NNP', 'NNG']:

In [38]:
stopwords_kr = []

with open("stopwordlist-kr.txt", "r") as ins:
    for line in ins:
        stopwords_kr.append(line.rstrip('\n'))

http://www.ultravioletanalytics.com/2016/11/18/tf-idf-basics-with-pandas-scikit-learn/

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [44]:
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger, stop_words=stopwords_kr, ngram_range=(2,5), min_df=.0025, max_df=.5)
vect_weights = vect.fit_transform(documents)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

In [45]:
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
225,모터쇼 꽃 레이싱,0.010344
174,꽃 레이싱,0.010344
202,레이싱 모델,0.010344
226,모터쇼 꽃 레이싱 모델,0.010344
224,모터쇼 꽃,0.010344
175,꽃 레이싱 모델,0.010344
192,대선 후보,0.010338
294,세월호 유가족,0.010265
227,목포 신항,0.008893
234,미수 습자,0.008776


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(tokenizer=noun_tagger, min_df=.0025, max_df=.5, ngram_range=(2,6))

cvec_weights = cvec.fit_transform(documents)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
cvec_weights = np.asarray(cvec_weights.mean(axis=0)).ravel().tolist()
cvecs_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': cvec_weights})

http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py

https://www.youtube.com/watch?v=znfy3T9OiAQ

In [49]:
cvecs_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
252,모터쇼 꽃,0.025641
228,레이싱 모델,0.025641
198,꽃 레이싱,0.025641
199,꽃 레이싱 모델,0.025641
253,모터쇼 꽃 레이싱,0.025641
254,모터쇼 꽃 레이싱 모델,0.025641
263,미수 습자,0.022624
72,4 월,0.021116
219,대선 후보,0.015083
90,8 명,0.013575


In [43]:
sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)

['후 태극기 집회',
 '후 태극기',
 '후 조사',
 '후 연락 두절',
 '후 연락',
 '회 로또',
 '황교안 권한 대행',
 '황교안 권한',
 '황 권한 대행',
 '황 권한',
 '화물선 침수 신고 후',
 '화물선 침수 신고',
 '화물선 침수',
 '화물선 양서',
 '홍준표 후보',
 '현충원 참배',
 '허용 증거 인멸 우려',
 '허용 증거 인멸',
 '허용 증거',
 '함안 군수',
 '한반도 전개',
 '한국인 8 명 화물선',
 '한국인 8 명 탑승',
 '한국인 8 명',
 '한국인 8',
 '한국 당은 게',
 '한국 당은',
 '한국 당 홍준표 후보',
 '한국 당 홍준표',
 '한국 당',
 '한 미 fta',
 '한 미',
 '폭격기 b 1 b',
 '폭격기 b 1',
 '폭격기 b',
 '폐업 애국',
 '펄 제거 작업',
 '펄 제거',
 '패션 위크',
 '트럼프 정부 무더기 대북',
 '트럼프 정부 무더기',
 '트럼프 정부 대북 제재',
 '트럼프 정부 대북',
 '트럼프 정부',
 '태극기 집회',
 '탑승 화물선',
 '침수 신고 후 연락',
 '침수 신고 후',
 '침수 신고',
 '척 발견 선원 구조',
 '척 발견 선원',
 '척 발견',
 '진해 군항 개막',
 '진해 군항',
 '진상 규명',
 '지지 선언',
 '지역 경선',
 '증거 인멸 우려 해소',
 '증거 인멸 우려',
 '증거 인멸',
 '주간 계획 4 3',
 '주간 계획 4',
 '주간 계획',
 '조사 위',
 '조사 검토',
 '제재 석탄',
 '제재 北 석탄 기업',
 '제재 北 석탄',
 '제재 北',
 '제거 작업',
 '정상 회담',
 '정부 무더기 대북 제재',
 '정부 무더기 대북',
 '정부 무더기',
 '정부 대북 제재',
 '정부 대북',
 '전략 폭격기',
 '적폐 청산',
 '자율 주행',
 '일반 면회 허용 증거',
 '일반 면회 허용',
 '일반 면회',
 '인멸 우려 해소',
 '인멸 우려',
 '윤 홍',

In [ ]:
to_csv